In [ ]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [1]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Text = data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm')

Label = data.LabelField(dtype = torch.float)

In [2]:
#The following code automatically downloads the IMDb dataset and splits it into the canonical train/test splits as torchtext.datasets objects
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(Text, Label)

train_data, test_data = datasets.IMDB.splits(Text, Label)

aclImdb_v1.tar.gz:   1%|          | 590k/84.1M [00:00<00:16, 5.10MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 80.0MB/s]


In [3]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [4]:
print(vars(train_data.examples[0]))

{'text': ['To', 'finally', 'see', 'what', 'many', 'consider', 'to', 'be', 'the', 'greatest', 'women', '-', 'in', '-', 'prison', 'film', 'of', 'all', 'time', ',', 'I', 'felt', 'like', 'I', 'had', 'accomplished', 'something', 'as', 'ridiculous', 'as', 'that', 'sounds', '.', 'Boy', ',', 'it', 'sure', 'contained', 'the', 'elements', 'I', 'expected', ',', 'and', 'delivered', 'so', 'much', 'more', '.', 'A', 'constant', 'I', "'m", 'discovering', 'in', 'these', 'films', 'is', 'the', 'toughness', 'and', 'grit', 'of', 'the', 'actresses', 'in', 'the', 'roles', 'of', 'prisoners', 'preparing', 'for', 'escape', 'while', 'their', 'threshold', ',', 'tolerance', ',', 'and', 'resolve(', '..', 'not', 'to', 'mention', 'sanity)being', 'tested', 'by', 'their', 'superiors', '.', 'While', 'most', 'of', 'them', 'were', 'hired', 'for', 'the', 'way', 'they', 'look', 'naked', ',', 'because', 'the', 'nature', 'of', 'the', 'genre', 'demands', 'such', 'gratuitous', 'elements', ',', 'something', 'else', 'emerges', ',

In [5]:
import random 

train_data, val_data = train_data.split(random_state = random.seed(SEED))

In [6]:
print(f"Number of Traning example: {len(train_data)}")
print(f"Number of Validation example: {len(val_data)}")
print(f"Number of Testing example: {len(test_data)}")

Number of Traning example: 17500
Number of Validation example: 7500
Number of Testing example: 25000


In [7]:
Max_Vocab_Size = 25_000

Text.build_vocab(train_data, max_size = Max_Vocab_Size)
Label.build_vocab(train_data)

In [8]:
print(f"Unique tokens in TEXT vocabulary: {len(Text.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(Label.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [9]:
#to check whihc is most commom words
print(Text.vocab.freqs.most_common(20))

[('the', 202434), (',', 191486), ('.', 165285), ('a', 109359), ('and', 109277), ('of', 100782), ('to', 93524), ('is', 76351), ('in', 61225), ('I', 54232), ('it', 53664), ('that', 49264), ('"', 44659), ("'s", 43164), ('this', 42454), ('-', 37317), ('/><br', 35633), ('was', 34935), ('as', 30466), ('with', 29838)]


In [10]:
print(Text.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [11]:
print(Label.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [12]:
Batch_Size = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, val_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, val_data, test_data), 
    batch_size = Batch_Size,
    device = device)

In [13]:
import torch.nn as nn

class RNN(nn.Module):

  def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
    super().__init__()

    self.embedding = nn.Embedding(input_dim, embedding_dim)
    self.rnn = nn.RNN(embedding_dim, hidden_dim)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, text):

    embedded = self.embedding(text)
    output,hidden = self.rnn(embedded)
    assert torch.equal(output[-1,:,:],hidden.squeeze(0))

    return self.fc(hidden.squeeze(0))

In [14]:
INPUT_dim = len(Text.vocab)
EMBEDDING_dim = 100
HIDDEN_dim= 256
OUTPUT_dim = 1

model = RNN(INPUT_dim, EMBEDDING_dim, HIDDEN_dim, OUTPUT_dim)

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [16]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [17]:
criterion = nn.BCEWithLogitsLoss()

In [18]:
model = model.to(device)
criterion = criterion.to(device)

In [19]:
def binary_accuracy(preds, y):

    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)

    return acc

In [20]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()     
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)



In [21]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:
import time

def epoch_time(start_time, end_time):
  
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))

    return elapsed_mins, elapsed_secs

In [24]:
NUM_epochs = 5

best_val_loss = float('inf')

for epoch in range(NUM_epochs):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 49.86%
	 Val. Loss: 0.696 |  Val. Acc: 50.47%
Epoch: 02 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 50.21%
	 Val. Loss: 0.696 |  Val. Acc: 50.65%
Epoch: 03 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 49.87%
	 Val. Loss: 0.696 |  Val. Acc: 49.77%
Epoch: 04 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 50.39%
	 Val. Loss: 0.696 |  Val. Acc: 50.75%
Epoch: 05 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 49.88%
	 Val. Loss: 0.696 |  Val. Acc: 49.79%


In [37]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.708 | Test Acc: 47.88%
